In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math

from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

%matplotlib inline

In [2]:
AdvWorksTest = pd.read_csv('AW_test.csv')
AdvWorksTest.columns = [str.replace('-', '_') for str in AdvWorksTest.columns]

for col in AdvWorksTest.columns:
    if AdvWorksTest[col].dtype == object:
        count = 0
        count = [count + 1 for x in AdvWorksTest[col] if x == '?']
        print(col + ' ' + str(sum(count)))
        
AdvWorksTest.shape      

Title 0
FirstName 0
MiddleName 0
LastName 0
Suffix 0
AddressLine1 0
AddressLine2 0
City 0
StateProvinceName 0
CountryRegionName 0
PostalCode 0
PhoneNumber 0
BirthDate 0
Education 0
Occupation 0
Gender 0
MaritalStatus 0


(500, 23)

In [3]:
AdvWorksTest.head(5)  

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,18988,NaN,Courtney,A,Baker,NaN,8727 Buena Vista Ave.,NaN,Fremont,California,...,1/5/1945,Bachelors,Management,F,S,0,2,0,5,86931
1,29135,NaN,Adam,C,Allen,NaN,3491 Cook Street,NaN,Haney,British Columbia,...,10/4/1964,Bachelors,Skilled Manual,M,M,1,2,2,4,100125
2,12156,NaN,Bonnie,NaN,Raji,NaN,359 Pleasant Hill Rd,NaN,Burbank,California,...,1/12/1934,Graduate Degree,Management,F,M,1,2,0,4,103985
3,13749,NaN,Julio,C,Alonso,NaN,8945 Euclid Ave.,NaN,Burlingame,California,...,9/22/1958,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161
4,27780,NaN,Christy,A,Andersen,NaN,"42, boulevard Tremblay",NaN,Dunkerque,Nord,...,3/19/1965,High School,Manual,F,M,1,1,2,2,21876


In [4]:
AdvWorksTest.drop('Title', axis = 1, inplace = True)
AdvWorksTest.drop('Suffix', axis = 1, inplace = True)
AdvWorksTest.drop('AddressLine2', axis = 1, inplace = True)
AdvWorksTest.drop('MiddleName', axis = 1, inplace = True)
AdvWorksTest.head(5)  

,CustomerID,FirstName,LastName,AddressLine1,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,18988,Courtney,Baker,8727 Buena Vista Ave.,Fremont,California,United States,94536,133-555-0128,1/5/1945,Bachelors,Management,F,S,0,2,0,5,86931
1,29135,Adam,Allen,3491 Cook Street,Haney,British Columbia,Canada,V2W 1W2,252-555-0173,10/4/1964,Bachelors,Skilled Manual,M,M,1,2,2,4,100125
2,12156,Bonnie,Raji,359 Pleasant Hill Rd,Burbank,California,United States,91502,409-555-0193,1/12/1934,Graduate Degree,Management,F,M,1,2,0,4,103985
3,13749,Julio,Alonso,8945 Euclid Ave.,Burlingame,California,United States,94010,175-555-0196,9/22/1958,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161
4,27780,Christy,Andersen,"42, boulevard Tremblay",Dunkerque,Nord,France,59140,1 (11) 500 555-0122,3/19/1965,High School,Manual,F,M,1,1,2,2,21876


In [5]:
from datetime import datetime

def age_customer(dl):
    Db = datetime.strptime(dl, '%m/%d/%Y')
    dl = datetime.strptime('1998-01-01', '%Y-%m-%d').year - Db.year
 
    if dl > 45: return 'Above 55'
    elif 45 < dl <= 55: return 'Between 45 and 55'
    elif 25 < dl <= 45: return 'Between 25 and 45'
    elif 0 < dl <= 25: return 'Under 25'
    else: return 'None'
        
AdvWorksTest['GroupCustomers']  = AdvWorksTest.BirthDate.apply (lambda row: age_customer(row))   
AdvWorksTest['BikeBuyer'] = ""

print(AdvWorksTest.shape)
AdvWorksTest.head(5)

(500, 21)


,CustomerID,FirstName,LastName,AddressLine1,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,...,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,GroupCustomers,BikeBuyer
0,18988,Courtney,Baker,8727 Buena Vista Ave.,Fremont,California,United States,94536,133-555-0128,1/5/1945,...,Management,F,S,0,2,0,5,86931,Above 55,
1,29135,Adam,Allen,3491 Cook Street,Haney,British Columbia,Canada,V2W 1W2,252-555-0173,10/4/1964,...,Skilled Manual,M,M,1,2,2,4,100125,Between 25 and 45,
2,12156,Bonnie,Raji,359 Pleasant Hill Rd,Burbank,California,United States,91502,409-555-0193,1/12/1934,...,Management,F,M,1,2,0,4,103985,Above 55,
3,13749,Julio,Alonso,8945 Euclid Ave.,Burlingame,California,United States,94010,175-555-0196,9/22/1958,...,Skilled Manual,M,M,1,0,0,4,127161,Between 25 and 45,
4,27780,Christy,Andersen,"42, boulevard Tremblay",Dunkerque,Nord,France,59140,1 (11) 500 555-0122,3/19/1965,...,Manual,F,M,1,1,2,2,21876,Between 25 and 45,


In [6]:
AdvWork_True = AdvWorksTest[['CustomerID','Education','Occupation','Gender','MaritalStatus'
                         ,'HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome','YearlyIncome'
                         ,'TotalChildren','GroupCustomers']]

print(AdvWork_True.shape)
print(AdvWork_True.CustomerID.unique().shape)
AdvWork_True.head(5)

(500, 11)
(500,)


,CustomerID,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,YearlyIncome,TotalChildren,GroupCustomers
0,18988,Bachelors,Management,F,S,0,2,0,86931,5,Above 55
1,29135,Bachelors,Skilled Manual,M,M,1,2,2,100125,4,Between 25 and 45
2,12156,Graduate Degree,Management,F,M,1,2,0,103985,4,Above 55
3,13749,Graduate Degree,Skilled Manual,M,M,1,0,0,127161,4,Between 25 and 45
4,27780,High School,Manual,F,M,1,1,2,21876,2,Between 25 and 45


In [7]:
AdvWork_True.describe()

,CustomerID,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,YearlyIncome,TotalChildren
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,20268.290000,0.666000,1.592000,1.150000,81795.474000,2.188000
std,5385.050297,0.472112,1.136715,1.596433,40627.641775,1.696589
min,11018.000000,0.000000,0.000000,0.000000,10588.000000,0.000000
25%,15464.250000,0.000000,1.000000,0.000000,50273.750000,1.000000
50%,20452.000000,1.000000,2.000000,0.000000,82298.000000,2.000000
75%,24789.000000,1.000000,2.000000,2.000000,107067.750000,4.000000
max,29483.000000,1.000000,4.000000,5.000000,184897.000000,5.000000


In [8]:
labels = np.array(AdvWork_True['YearlyIncome'])
print(labels)

[ 86931 100125 103985 127161  21876  44467  77702  99418  13522  46264
  96375  77688  52127  60978  36915  86445 113658  83326 142823  42704
  31323  61350 148087 112248  94217 122711  58716 128140 101320 150227
  85726 125452  50463  91982 135619  62347 157083  92155 111592  61015
  86042  16149  89009  71664  54946  61296  26298 121950 165094  87346
 122359  80468 131206  20427  63991  63830  66343  86825  17410  57913
  48202  19745  88827 101507  23910  92860 121541 141619  39565 139299
  82733  66790 102700  46284  56823 102763  63530  38524 122214  58812
  96408  80326  65596 170887  57415  19824 108839 163183  40146 107111
 113337  81034  72787 102945  20560 161140  16588  73662  57023  89625
  89770 118636  71702 154515 114039  98023  21984 107067  87337 143858
  84569  92946  83958  39320  61300  93527  46297  53441  88333  98760
  86846 100001 122694  31139 101616  61458  47048 140321  40997 174134
  40926  18156  64285  71487  73600 135451 165902 103153  88798 101566
 12799

In [9]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Occupation','Gender','MaritalStatus']

Features = encode_string(AdvWork_True['Education'])
for col in categorical_columns:
    temp = encode_string(AdvWork_True[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)
print(Features[:2, :])   

(500, 14)
[[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0.]]


In [10]:
AdvWork_MM = np.concatenate([Features, np.array(AdvWork_True[['NumberCarsOwned','NumberChildrenAtHome'
                                                              ,'YearlyIncome','TotalChildren']])], axis = 1)
print(AdvWork_MM.shape)
print(AdvWork_MM[:2, :])   

(500, 18)
[[1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00
  0.00000e+00 1.00000e+00 2.00000e+00 0.00000e+00 8.69310e+04 5.00000e+00]
 [1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00 1.00000e+00
  1.00000e+00 0.00000e+00 2.00000e+00 2.00000e+00 1.00125e+05 4.00000e+00]]


In [11]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(AdvWork_MM.shape[0])
indx = ms.train_test_split(indx, test_size = 300)

X_train = AdvWork_MM[indx[0],:]
y_train = np.ravel(labels[indx[0]])
X_test = AdvWork_MM[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [12]:
scaler = preprocessing.StandardScaler().fit(X_train[:,17:])
X_train[:,17:] = scaler.transform(X_train[:,17:])
X_test[:,17:] = scaler.transform(X_test[:,17:])
X_train[:2,]

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         2.00000000e+00,  7.75980000e+04, -9.91787324e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  2.00000000e+00,
         3.00000000e+00,  4.58850000e+04,  5.01904494e-01]])

In [13]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
print(logistic_mod.intercept_)
print(logistic_mod.coef_)

[-1.43311682e-09  4.82726206e-02  3.61782005e-06  2.17609110e-06
  2.62867995e-06  1.61514177e-06  1.83271729e-06  1.13689443e-06
 -5.24571791e-07 -6.63745888e-07 -9.82069498e-07 -5.19005581e-09
 -5.19030514e-09 -5.38149290e-09 -5.42584838e-09 -5.44560761e-09
 -5.45243338e-09 -5.45713727e-09 -5.58633730e-09 -5.64915894e-09
 -5.76436780e-09 -5.87303268e-09 -5.92167815e-09 -3.74794500e-02
 -6.15530404e-09 -6.19503634e-09 -6.21780483e-09 -6.22816211e-09
 -6.27154085e-09 -6.28387743e-09 -6.32661989e-09 -6.33105622e-09
 -6.34412555e-09 -6.35197912e-09 -3.26152322e-01 -9.43682507e-06
 -2.95572559e-01 -1.02534786e-02 -1.49380169e-01 -7.87074491e-06
 -7.98036873e-06 -8.75069670e-06 -8.50301968e-06 -3.59818858e-01
 -8.50240738e-06 -8.89091901e-06 -1.09700547e-05 -3.35445426e-01
 -3.09385144e-01 -3.84009585e-01 -9.07581025e-06 -3.32956751e-01
 -3.09509898e-03 -4.13528800e-01 -1.24517856e-02 -4.40286120e-01
 -5.35889976e-01 -3.72902198e-01 -5.25945664e-01 -5.04387338e-01
 -4.37821067e-01 -4.34161

In [15]:
probabilities = logistic_mod.predict_proba(X_test)
print(probabilities[:15,15:])

[[6.07822851e-04 6.11373035e-04 6.13836377e-04 ... 7.63590269e-03
  4.84917647e-03 9.40025366e-03]
 [3.70461287e-10 3.78774164e-10 3.84622493e-10 ... 7.68135475e-02
  1.50899281e-02 5.65712886e-02]
 [1.82999338e-03 1.83655538e-03 1.84109975e-03 ... 2.91766510e-03
  8.30425555e-03 4.16408485e-03]
 ...
 [8.77614436e-07 8.89410318e-07 8.97647323e-07 ... 2.44061887e-02
  6.10048272e-03 1.42284713e-02]
 [1.32247226e-05 1.33557592e-05 1.34469916e-05 ... 1.05067074e-02
  1.50414359e-02 1.59235953e-02]
 [4.39932787e-08 4.46655275e-08 4.51356743e-08 ... 2.46104283e-02
  2.02995836e-02 2.80721921e-02]]


In [16]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])
scores = score_model(probabilities, 0.80)
print(np.array(scores))
print(y_test)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
[ 45972 174979  28631  81160  83382  48123 102964  91953  80468 102763
 131508  30824 105276  77745 119578  62337  40997  18156 136345  65596
  50921 101507  23449  51042  68194  42318  27771 167054 112760 106878
 101162 133419 100001  38145  86037  34609 135829  81701  46556  99418
  98371  82733  94217  65238  84815  47048  92753  69562 141619 141543
 123254 114039  39565  22875  75962

In [17]:
def print_metrics(labels, scores):
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy  %0.2f' % sklm.accuracy_score(labels, scores))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])

  
print_metrics(y_test, scores)     

                 Confusion matrix
                 Score positive    Score negative
Actual positive         0                 0
Actual negative         1                 0

Accuracy  0.00
 
           Positive      Negative
Num case        0             1
Precision    0.00          0.00
Recall       0.00          0.00
F1           0.00          0.00


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
